In [4]:
import os
os.chdir("C:/Users/Anushka/Desktop/anushka/ML/projects_vscode/IPL prediction system")


Adding XGBoost, LightGBM, and a Neural Network model, and preparing for an ensemble.


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import joblib

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical




In [16]:
df = pd.read_csv("data_processing/processed_data/match_level_data.csv")

team_encoder = LabelEncoder()
df['team1_encoded'] = team_encoder.fit_transform(df['team1'])

# Define features and target
X = df[['team1_encoded', 'team1_score', 'team1_wickets']]
y = df['winner_encoded']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Scale features for NN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# XGBoost Model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

# LightGBM Model
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train, y_train)
lgbm_preds = lgbm_model.predict(X_test)

# Neural Network Model
nn_model = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
nn_model.fit(X_train_scaled, y_train)
nn_preds = nn_model.predict(X_test_scaled)



c:\Users\Anushka\Desktop\anushka\ML\projects_vscode\IPL prediction system\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:53:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000765 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 151
[LightGBM] [Info] Number of data points in the train set: 872, number of used features: 3
[LightGBM] [Info] Start training from score -2.145817
[LightGBM] [Info] Start training from score -3.474953
[LightGBM] [Info] Start training from score -3.081910
[LightGBM] [Info] Start training from score -2.745438
[LightGBM] [Info] Start training from score -4.691348
[LightGBM] [Info] Start training from score -3.826350
[LightGBM] [Info] Start training from score -2.480330
[LightGBM] [Info] Start training from score -5.384495
[LightGBM] [Info] Start training from score -2.145817
[LightGBM] [Info] Start training from score -3.726267
[LightGBM] [Info] Start training from score -2.008615
[LightGBM] [Info] Start training from score -4.468204
[Li

c:\Users\Anushka\Desktop\anushka\ML\projects_vscode\IPL prediction system\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [18]:
# Evaluation
def evaluate(name, y_true, y_pred):
    print(f"{name} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{name} F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
    print()

evaluate("XGBoost", y_test, xgb_preds)
evaluate("LightGBM", y_test, lgbm_preds)
evaluate("Neural Network", y_test, nn_preds)

XGBoost Accuracy: 0.4450
XGBoost F1 Score: 0.4365

LightGBM Accuracy: 0.4266
LightGBM F1 Score: 0.4189

Neural Network Accuracy: 0.4908
Neural Network F1 Score: 0.4580



In [21]:
import pickle
# Save models and encoders
pickle.dump(xgb_model, open("ml_model/xgb_model.pkl", "wb"))
pickle.dump(lgbm_model, open("ml_model/lgbm_model.pkl", "wb"))
pickle.dump(nn_model, open("ml_model/nn_model.pkl", "wb"))
pickle.dump(scaler, open("ml_model/scaler.pkl", "wb"))
pickle.dump(team_encoder, open("ml_model/team_encoder.pkl", "wb"))